# Total Employment Trend in Franklin and Delaware Countires

## Introduction

This script is meant to take Quarterly Census of Employment and Wages data to filter for Franklin and Delaware Counties from 2010-2021, and create a new combined table and line graph to visualize the employment levels in both counties.

### Process Outline

The process carried out by this workflow can be described as follows:
  - The script will retrieve the QCEW datasets from the US Bureau of Labor Statistics
  - Filter to only retain and sum records from Franklin or Delawre, Ohio 2010-2021
  - Identify and keep only the rows containing all industires and all coverage
  - Write aggregated table and line chart to Excel to visualize the changes in employment levels

## Setup

### Import required package

In [72]:
import csv
import os
import urllib.request
import urllib
import pandas as pd
import json
from tableschema import Table

### Parameters

In [43]:
# Define input and output directories
INPUT_DIR = "./input_data"
OUTPUT_DIR = "./output_data"

### Define inputs

#### QCEW data for Franklin and Delaware Counties

In [84]:
COMBINED_TABLE_FILENAME = "combined.csv"
COMBINED_TABLE_PATH = os.path.join(INPUT_DIR, COMBINED_TABLE_FILENAME)
COMBINED_TABLE_SCHEMA_FILENAME = COMBINED_TABLE_FILENAME.replace(".csv","_schema.json")
COMBINED_TABLE_SCHEMA_PATH = os.path.join(INPUT_DIR, COMBINED_TABLE_SCHEMA_FILENAME)
print("Data: {}".format(COMBINED_TABLE_PATH))
print("Schema: {}".format(COMBINED_TABLE_SCHEMA_PATH))

Data: ./input_data\combined.csv
Schema: ./input_data\combined_schema.json


#### Complined Excel Sheet

In [89]:
EXCEL_TABLE_FILENAME = "compiled.xlsx"
EXCEL_TABLE_PATH = os.path.join(OUTPUT_DIR, EXCEL_TABLE_FILENAME)
print("Output data path: {}".format(EXCEL_TABLE_PATH))

Output data path: ./output_data\compiled.xlsx


### Define outputs

#### Franklin and Delaware counties employment level by year (2010-2021)

In [108]:
EMPLVL_TABLE_FILENAME = "annual_avg_emplvl.csv"
EMPLVL_TABLE_PATH = os.path.join(OUTPUT_DIR, EMPLVL_TABLE_FILENAME)
EMPLVL_TABLE_SCHEMA_FILENAME = EMPLVL_TABLE_FILENAME.replace(".csv","_schema.json")
EMPLVL_TABLE_SCHEMA_PATH = os.path.join(OUTPUT_DIR, EMPLVL_TABLE_SCHEMA_FILENAME)
print("Data: {}".format(EMPLVL_TABLE_PATH))
print("Schema: {}".format(EMPLVL_TABLE_SCHEMA_PATH))

Data: ./output_data\annual_avg_emplvl.csv
Schema: ./output_data\annual_avg_emplvl_schema.json


## Getting input data

In [98]:
# *******************************************************************************
# qcewCreateDataRows : This function takes a raw csv string and splits it into
# a two-dimensional array containing the data and the header row of the csv file
# a try/except block is used to handle for both binary and char encoding
def qcewCreateDataRows(csv):
    dataRows = []
    try: dataLines = csv.decode().split('\r\n')
    except er: dataLines = csv.split('\r\n');
    for row in dataLines:
        dataRows.append(row.split(','))
    return dataRows
# *******************************************************************************

# *******************************************************************************
# qcewGetAreaData : This function takes a year, quarter, and area argument and
# returns an array containing the associated area data. Use 'a' for annual
# averages. 
# For all area codes and titles see:
# http://www.bls.gov/cew/doc/titles/area/area_titles.htm
#
def qcewGetAreaData(year,qtr,area):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[AREA]",area.upper())
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)


def fetch_and_combine_qcew_data(start_year, end_year, qtr, area):
    all_years_data = []
    for year in range(start_year, end_year + 1):
        print(f"Fetching data for {year}...")
        year_data = qcewGetAreaData(str(year), qtr, area)
        columns_no_quotes = [col.replace('"', '') for col in year_data[0]]
        year_data_df = pd.DataFrame(year_data[1:], columns=columns_no_quotes)
        # Add a column to distinguish data by year
        year_data_df['year'] = year
        # Remove every instance of " in the data
        year_data_df = year_data_df.replace('"', '', regex=True)
        all_years_data.append(year_data_df)
        
    # Combine all DataFrame objects into a single DataFrame
    combined_df = pd.concat(all_years_data, ignore_index=True)
    
    # Apply filtering: retain rows where industry_code == "10" and own_code == "0"
    filtered_df = combined_df.loc[(combined_df['industry_code'] == "10") & (combined_df['own_code'] == "0")]

    
    return filtered_df

# Fetch and combine the data
combined_franklin_data = fetch_and_combine_qcew_data(2014, 2021, "a", "39049")

# Fetch and combine the data
combined_delaware_data = fetch_and_combine_qcew_data(2014, 2021, "a", "39041")

# Combine the DataFrames
combined_df = pd.concat([combined_franklin_data, combined_delaware_data], ignore_index=True)

combined_df.to_csv(COMBINED_TABLE_PATH, index=False)

print("The CSV files have been combined and saved.")
print(f"Processed data saved to {COMBINED_TABLE_PATH}")


# Create table
table = Table(COMBINED_TABLE_PATH)

# Infer table variable types
table.infer()

# Convert missing values to 'N/A'
table.schema.descriptor['missingValues'] = ['N/A', '']
table.schema.commit()

# Save schema
table.schema.save(COMBINED_TABLE_SCHEMA_PATH)



# Load the schema from the JSON file
with open(COMBINED_TABLE_SCHEMA_PATH, 'r') as file:
    schema = json.load(file)

# Define a function to convert data types based on the schema
def convert_data_types(row, schema):
    converted_row = {}
    for field in schema['fields']:
        field_name = field['name']
        field_type = field['type']
        if field_name in row:
            if field_type == 'int':
                converted_row[field_name] = int(row[field_name])
            elif field_type == 'string':
                converted_row[field_name] = str(row[field_name])
            # Add more type conversions as needed
            else:
                converted_row[field_name] = row[field_name]  # Keep as is if type not recognized
    return converted_row

# Read the input data, apply conversion, and process it
converted_data = []

with open(COMBINED_TABLE_PATH, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        converted_row = convert_data_types(row, schema)
        converted_data.append(converted_row)

df = pd.read_csv(COMBINED_TABLE_PATH)
df.head()

# Aggregate rows by "year" and sum up all other numeric columns
aggregated_df = df.groupby('year').sum().reset_index()

aggregated_df.to_csv(COMBINED_TABLE_PATH, index=False)

print("The data has been aggregated by year and saved.")

Fetching data for 2014...
Fetching data for 2015...
Fetching data for 2016...
Fetching data for 2017...
Fetching data for 2018...
Fetching data for 2019...
Fetching data for 2020...
Fetching data for 2021...
Fetching data for 2014...
Fetching data for 2015...
Fetching data for 2016...
Fetching data for 2017...
Fetching data for 2018...
Fetching data for 2019...
Fetching data for 2020...
Fetching data for 2021...
The CSV files have been combined and saved.
Processed data saved to ./input_data\combined.csv
The data has been aggregated by year and saved.


C:\Users\ogwynn\AppData\Local\Temp\ipykernel_11564\3004708976.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['annual_avg_emplvl_thousands'] = filtered_df['annual_avg_emplvl'] / 1000


## Formatting and fitlering data

In [109]:
# Read the CSV file into a DataFrame
df = pd.read_csv(COMBINED_TABLE_PATH)
df.head()

# Select only the 'year' and 'annual_avg_emplvl' columns
data = df[['year', 'annual_avg_emplvl']]

data.to_csv(EMPLVL_TABLE_PATH, index=False)



# Create table
table = Table(EMPLVL_TABLE_PATH)

# Infer table variable types
table.infer()

# Convert missing values to 'N/A'
table.schema.descriptor['missingValues'] = ['N/A', '']
table.schema.commit()

# Save schema
table.schema.save(EMPLVL_TABLE_SCHEMA_PATH)

## Dataset and graph to Excel

In [104]:
def create_line_chart_excel(df, excel_file_path):
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Sheet1', index=False)
        legend_label = 'Total Employment'
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']
        
        chart = workbook.add_chart({'type': 'line'})
        
        chart.add_series({
            'name': legend_label,
            'categories': '=Sheet1!$A$2:$A${}'.format(len(df) + 1),
            'values': '=Sheet1!$C$2:$C${}'.format(len(df) + 1),
        })
        
        # Set chart title with Arial font
        chart.set_title({
            'name': 'Total Employment Trend',
            'name_font': {'name': 'Arial', 'size': 14}
        })
        
        # Set x-axis with Arial font and remove tick marks and line
        chart.set_x_axis({
            'name': '',
            'name_font': {'name': 'Arial', 'size': 8},  # Axis title font
            'num_font': {'name': 'Arial', 'size': 8},   # Tick labels font
            'major_tick_mark': 'none',
            'line': {'none': True}
        })
        
        # Set y-axis with Arial font and remove the line
        chart.set_y_axis({
            'name': 'Thousands',
            'name_font': {'name': 'Arial', 'size': 8},  # Axis title font
            'num_font': {'name': 'Arial', 'size': 8},   # Tick labels font
            'major_gridlines': {
            'visible': True, 
            'line': {'color': '#808080', 'width': 1}  # Medium gray color for major gridlines
        },
        'line': {'none': True}  # Remove y-axis line
    })

        chart.set_legend({'position': 'bottom', 'font': {'name': 'Arial', 'size': 10}})

        # Insert the chart into the worksheet
        worksheet.insert_chart('D2', chart)


# Create the line chart in the Excel file
create_line_chart_excel(filtered_df, EXCEL_TABLE_PATH)

print("Excel file with line chart created successfully.")

Excel file with line chart created successfully.
